### Configuración Inicial

In [1]:
!pip install deepctr-torch torch pandas numpy scikit-learn recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 17.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.3/355.3 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.6/29.6 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.9/295.9 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.0/115.0 kB 14.1 MB/s eta

### Instalación de Librerías

In [2]:
import pandas as pd
import numpy as np
import gdown
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import torch
from deepctr_torch.inputs import SparseFeat, get_feature_names
from deepctr_torch.models import DeepFM

### Importación de los Datos

In [3]:
gdown.download(id='1eGDDR1wlvR99eoCZG2owChy2dhkPp4yx', output='training_ratings.csv', quiet=False)
gdown.download(id='1oHo9HLB6SzeqZs76FCkfQ1irSQepqp16', output='validation_ratings.csv', quiet=False)

# dataset mechanics
gdown.download(id='1cVGSLNVqxrAoKzeqxt_FfQ4Ggs9VvCDO', output='mechanics.csv', quiet=False)
df_mechanics = pd.read_csv('mechanics.csv')

df_train = pd.read_csv('training_ratings.csv')
df_val = pd.read_csv('validation_ratings.csv')

df_full = pd.concat([df_train, df_val])


Downloading...
From (original): https://drive.google.com/uc?id=1eGDDR1wlvR99eoCZG2owChy2dhkPp4yx
From (redirected): https://drive.google.com/uc?id=1eGDDR1wlvR99eoCZG2owChy2dhkPp4yx&confirm=t&uuid=3e56242c-0a5c-4298-aa3e-a6eb85a9b406
To: /content/training_ratings.csv
100%|██████████| 205M/205M [00:03<00:00, 66.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1oHo9HLB6SzeqZs76FCkfQ1irSQepqp16
To: /content/validation_ratings.csv
100%|██████████| 64.4M/64.4M [00:01<00:00, 36.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1cVGSLNVqxrAoKzeqxt_FfQ4Ggs9VvCDO
To: /content/mechanics.csv
100%|██████████| 7.05M/7.05M [00:00<00:00, 23.9MB/s]


### Preprocesamiento de Datos

In [4]:
df_train.drop_duplicates(inplace=True, subset=['user', 'item'])
df_val.drop_duplicates(inplace=True, subset=['user', 'item'])

In [5]:
print(f"Tamaño original del training set: {len(df_train)}")

# se obtiene un sample debido a que hay muchos datos y se demora mucho
df_train_sample = df_train.sample(n=10000, random_state=42)
print(f"Tamaño del nuevo training set (muestra): {len(df_train_sample)}")


Tamaño original del training set: 10200445
Tamaño del nuevo training set (muestra): 10000


In [6]:
df_mechanics.set_index('BGGId', inplace=True)
print("Datos de mecánicas listos con 'BGGId' como índice.")
print(df_mechanics.head())

Datos de mecánicas listos con 'BGGId' como índice.
       Alliances  Area Majority / Influence  Auction/Bidding  Dice Rolling  \
BGGId                                                                        
1              1                          1                1             1   
2              0                          0                0             0   
3              0                          1                0             0   
4              0                          1                1             0   
5              0                          0                0             0   

       Hand Management  Simultaneous Action Selection  Trick-taking  \
BGGId                                                                 
1                    1                              1             0   
2                    0                              0             1   
3                    1                              0             0   
4                    0                         

In [7]:
item_popularity = df_train['item'].value_counts().to_dict()
total_interactions = len(df_train)

# Convertir la cuenta de popularidad en una probabilidad para el cálculo de novedad
item_popularity_prob = {item_id: count / total_interactions for item_id, count in item_popularity.items()}

print(f"Se calculó la popularidad para {len(item_popularity)} ítems.")

Se calculó la popularidad para 16748 ítems.


### Configuración de Experimentos

In [8]:
# Renombrar columnas
df_train_sample = df_train_sample.rename(columns={'user': 'userID', 'item': 'itemID', 'name': 'name'})
df_val = df_val.rename(columns={'user': 'userID', 'item': 'itemID'})

# Ratings >= 7 se consideran positivos (1), el resto negativos (0)
df_train_sample['label'] = (df_train_sample['rating'] >= 7).astype(int)
df_val['label'] = (df_val['rating'] >= 7).astype(int)

# Se usan las columnas 'userID' y 'itemID' de ambos dataframes para asegurar que todos los IDs estén en el codificador
sparse_features = ['userID', 'itemID']
all_data = pd.concat([df_train_sample, df_val], sort=False)

# Creamos un diccionario para guardar los codificadores
encoders = {}
for feat in sparse_features:
    lbe = LabelEncoder()
    all_data[feat] = lbe.fit_transform(all_data[feat])
    encoders[feat] = lbe

# Separar de nuevo en train y validation
df_train_processed = all_data.iloc[:len(df_train_sample)]
df_val_processed = all_data.iloc[len(df_train_sample):]

# Guardamos el codificador de itemID en una variable separada para fácil acceso
item_id_encoder = encoders['itemID']

print("\nEjemplo de datos procesados:")
print(df_train_processed.head())


Ejemplo de datos procesados:
         itemID  rating  userID  label
7635838    7927     6.0  189877      0
5061742   11971     8.0  129683      1
9066677      10     9.0  224036      1
2587984    9562     7.0   66546      1
4075764    9873     8.0  104440      1


### Predicción de ratings y top N



In [9]:
n_users = all_data['userID'].nunique()
n_items = all_data['itemID'].nunique()

print(f"Número de usuarios únicos: {n_users}")
print(f"Número de ítems únicos: {n_items}")


# Definir las características de entrada para el modelo
embedding_dim = 16 # Dimensión de los vectores latentes (embeddings)

feature_columns = [
    SparseFeat('userID', vocabulary_size=n_users, embedding_dim=embedding_dim),
    SparseFeat('itemID', vocabulary_size=n_items, embedding_dim=embedding_dim)
]

dnn_feature_columns = feature_columns
linear_feature_columns = feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

Número de usuarios únicos: 250682
Número de ítems únicos: 16744


In [10]:
# Dividir los datos de validación para tener un conjunto de testeo
train_model_input = {name: df_train_processed[name].values for name in feature_names}
train_labels = df_train_processed['label'].values

val_model_input = {name: df_val_processed[name].values for name in feature_names}
val_labels = df_val_processed['label'].values

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = DeepFM(
    linear_feature_columns=linear_feature_columns,
    dnn_feature_columns=dnn_feature_columns,
    task='binary',
    l2_reg_embedding=1e-5,
    device=device
)

model.compile(
    "adam",
    "binary_crossentropy",
    metrics=["binary_accuracy", "auc"],
)

In [11]:
history = model.fit(
    train_model_input,
    train_labels,
    batch_size=2048,
    epochs=10,
    verbose=2,
    validation_data=(val_model_input, val_labels)
)

print("\n¡Entrenamiento completado!")

cuda
Train on 10000 samples, validate on 3202256 samples, 5 steps per epoch
Epoch 1/10
16s - loss:  0.6838 - auc:  0.6092 - val_auc:  0.6613
Epoch 2/10
14s - loss:  0.6602 - auc:  0.9572 - val_auc:  0.6724
Epoch 3/10
15s - loss:  0.6419 - auc:  0.9754 - val_auc:  0.6735
Epoch 4/10
14s - loss:  0.6252 - auc:  0.9762 - val_auc:  0.6731
Epoch 5/10
15s - loss:  0.6123 - auc:  0.9762 - val_auc:  0.6717
Epoch 6/10
14s - loss:  0.6025 - auc:  0.9770 - val_auc:  0.6699
Epoch 7/10
14s - loss:  0.5899 - auc:  0.9785 - val_auc:  0.6671
Epoch 8/10
15s - loss:  0.5710 - auc:  0.9801 - val_auc:  0.6651
Epoch 9/10
14s - loss:  0.5460 - auc:  0.9818 - val_auc:  0.6628
Epoch 10/10
14s - loss:  0.5146 - auc:  0.9835 - val_auc:  0.6603

¡Entrenamiento completado!


In [12]:
from sklearn.metrics import ndcg_score
from sklearn.metrics.pairwise import cosine_similarity

pred_scores = model.predict(val_model_input, batch_size=2048)

df_eval = pd.DataFrame({
    'userID': df_val_processed['userID'].values,
    'itemID': df_val_processed['itemID'].values,
    'label': val_labels,
    'score': pred_scores.flatten(),
})



def precision_recall_at_k(group, k):
    """Calcula Precision@K y Recall@K para un solo usuario/grupo."""
    group = group.sort_values('score', ascending=False)
    topk = group.head(k)

    hits = topk['label'].sum()
    total_relevant = group['label'].sum()

    precision = hits / k
    recall = hits / total_relevant if total_relevant > 0 else 0

    return precision, recall

def ndcg_at_k(group, k):
    """Calcula nDCG@K para un solo usuario/grupo."""
    if group['label'].sum() == 0:
        return 0.0

    ranked_group = group.sort_values('score', ascending=False).head(k)

    # nDCG no se puede calcular si hay menos de 2 ítems en la lista.
    if len(ranked_group) < 2:
        return 0.0

    true_relevance = np.asarray([ranked_group['label'].values])
    predicted_scores = np.asarray([ranked_group['score'].values])

    return ndcg_score(true_relevance, predicted_scores)

def novelty_at_k(group, k, popularity_prob):
    """Calcula la Novedad@K para un solo usuario/grupo."""
    group = group.sort_values('score', ascending=False)
    # Usa la columna 'itemID' directamente
    topk_items = group.head(k)['itemID']

    novelty_scores = []
    for item_id in topk_items:
        # Usar una probabilidad pequeña si el ítem no se vio en el entrenamiento
        prob = popularity_prob.get(item_id, 1e-6)
        novelty_scores.append(-np.log2(prob))

    return np.mean(novelty_scores) if novelty_scores else 0.0

def diversity_at_k(group, k, mechanics_df):
    """Calcula la Diversidad@K (Intra-List Diversity) para un solo usuario/grupo."""
    group = group.sort_values('score', ascending=False)
    # Usa la columna 'itemID' directamente
    topk_items = group.head(k)['itemID'].tolist()

    # Filtra los ítems para asegurar que existan en el dataframe de mecánicas
    # Busca los 'itemID' en el índice de mechanics_df, que es 'BGGId'
    topk_items = [item for item in topk_items if item in mechanics_df.index]

    if len(topk_items) < 2:
        return 0.0

    # Obtiene los vectores de características (mecánicas) para los ítems recomendados
    item_vectors = mechanics_df.loc[topk_items].values

    # Calcula la disimilitud del coseno (1 - similitud) entre todos los pares de ítems
    dissimilarity_sum = 0
    num_pairs = 0
    for i in range(len(item_vectors)):
        for j in range(i + 1, len(item_vectors)):
            sim = cosine_similarity([item_vectors[i]], [item_vectors[j]])[0][0]
            dissimilarity_sum += (1 - sim)
            num_pairs += 1

    return dissimilarity_sum / num_pairs if num_pairs > 0 else 0.0


K_values = [10]
results = []

print("Calculando métricas de ranking para recomendaciones individuales...")

grouped = df_eval.groupby('userID')

for k in K_values:
    metrics = grouped.apply(lambda x: precision_recall_at_k(x, k))
    avg_precision = np.mean([m[0] for m in metrics])
    avg_recall = np.mean([m[1] for m in metrics])

    ndcg_scores = grouped.apply(lambda x: ndcg_at_k(x, k))
    avg_ndcg = np.mean(ndcg_scores)

    avg_novelty = grouped.apply(lambda x: novelty_at_k(x, k, item_popularity_prob)).mean()
    avg_diversity = grouped.apply(lambda x: diversity_at_k(x, k, df_mechanics)).mean()

    results.append({
        'K': k,
        'Precision@K': avg_precision,
        'Recall@K': avg_recall,
        'nDCG@K': avg_ndcg,
        'Novelty@K': avg_novelty,
        'Diversity@K': avg_diversity
    })

results_df = pd.DataFrame(results)
print("\n--- Resultados de Evaluación (DeepFM Individual) ---")
print(results_df)

Calculando métricas de ranking para recomendaciones individuales...


/tmp/ipython-input-3938952035.py:94: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics = grouped.apply(lambda x: precision_recall_at_k(x, k))
/tmp/ipython-input-3938952035.py:98: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_scores = grouped.apply(lambda x: ndcg_at_k(x, k))
/tmp/ipython-input-3938952035.py:101: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavi


--- Resultados de Evaluación (DeepFM Individual) ---
    K  Precision@K  Recall@K    nDCG@K  Novelty@K  Diversity@K
0  10     0.460928  0.829971  0.821499  18.706848     0.544632


/tmp/ipython-input-3938952035.py:102: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_diversity = grouped.apply(lambda x: diversity_at_k(x, k, df_mechanics)).mean()


In [13]:
# --- 1. Creación de Grupos Sintéticos ---

user_counts = df_eval['userID'].value_counts()
valid_users = user_counts[user_counts >= 10].index.tolist()

# Creamos 1000 grupos sintéticos de 3 usuarios cada uno
np.random.seed(42)
num_groups = 1000
group_size = 4
groups = [np.random.choice(valid_users, group_size, replace=False) for _ in range(num_groups)]

print(f"Se crearon {len(groups)} grupos sintéticos de tamaño {group_size}.")
print("Ejemplo de un grupo:", groups[0])


all_group_recs = []

for group_id, user_ids in enumerate(groups):
    group_predictions = df_eval[df_eval['userID'].isin(user_ids)]

    item_scores_per_group = group_predictions.groupby('itemID').agg(
        avg_score=('score', 'mean'),
        min_score=('score', 'min'),
        max_score=('score', 'max'),
        # Para el 'ground truth' del grupo, consideramos un ítem relevante
        # si A TODOS los miembros les gustó (label=1).
        # El producto de las etiquetas será 1 solo si todas son 1.
        group_label=('label', lambda x: 1 if all(x == 1) else 0)
    ).reset_index()

    item_scores_per_group['group_id'] = group_id
    all_group_recs.append(item_scores_per_group)

df_group_eval = pd.concat(all_group_recs, ignore_index=True)

print("\nEjemplo de scores agregados para un ítem y un grupo:")
print(df_group_eval.head())

Se crearon 1000 grupos sintéticos de tamaño 4.
Ejemplo de un grupo: [ 42457 221699 237772  29441]

Ejemplo de scores agregados para un ítem y un grupo:
   itemID  avg_score  min_score  max_score  group_label  group_id
0     358   0.748483   0.748483   0.748483            1         0
1     409   0.776132   0.776132   0.776132            0         0
2     566   0.792377   0.792377   0.792377            1         0
3     602   0.716277   0.716277   0.716277            0         0
4     895   0.620412   0.620412   0.620412            0         0


In [14]:
# --- Evaluar cada estrategia de agregación ---
strategies = {
    'Average': 'avg_score',
    'Least Misery': 'min_score',
    'Most Pleasure': 'max_score'
}

group_results = []

for strategy_name, score_column in strategies.items():
    print(f"Evaluando estrategia grupal: {strategy_name}...")

    df_strategy_eval = df_group_eval[['group_id', 'itemID', 'group_label']].copy()
    df_strategy_eval.rename(columns={'group_label': 'label'}, inplace=True)
    df_strategy_eval['score'] = df_group_eval[score_column]

    grouped_strategy = df_strategy_eval.groupby('group_id')

    for k in K_values:
        # Métricas existentes
        metrics = grouped_strategy.apply(lambda x: precision_recall_at_k(x, k))
        avg_precision = np.mean([m[0] for m in metrics])
        avg_recall = np.mean([m[1] for m in metrics])

        ndcg_scores = grouped_strategy.apply(lambda x: ndcg_at_k(x, k))
        avg_ndcg = np.mean(ndcg_scores)

        # Nuevas métricas de Novedad y Diversidad
        avg_novelty = grouped_strategy.apply(lambda x: novelty_at_k(x, k, item_popularity_prob)).mean()
        avg_diversity = grouped_strategy.apply(lambda x: diversity_at_k(x, k, df_mechanics)).mean()

        group_results.append({
            'Strategy': strategy_name,
            'K': k,
            'Precision@K': avg_precision,
            'Recall@K': avg_recall,
            'nDCG@K': avg_ndcg,
            'Novelty@K': avg_novelty,
            'Diversity@K': avg_diversity
        })

group_results_df = pd.DataFrame(group_results)

print("\n--- Resultados de Evaluación (Estrategias Grupales) ---")
print(group_results_df)

print("\n--- Comparativa Final: Individual vs. Grupal (nDCG@10) ---")
ndcg_individual = results_df[results_df['K'] == 10]['nDCG@K'].iloc[0]
print(f"DeepFM Individual: {ndcg_individual:.4f}")

for strategy_name in strategies.keys():
    ndcg_group = group_results_df[(group_results_df['Strategy'] == strategy_name) & (group_results_df['K'] == 10)]['nDCG@K'].iloc[0]
    print(f"{strategy_name}: {ndcg_group:.4f}")

Evaluando estrategia grupal: Average...


/tmp/ipython-input-2408301555.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics = grouped_strategy.apply(lambda x: precision_recall_at_k(x, k))
/tmp/ipython-input-2408301555.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_scores = grouped_strategy.apply(lambda x: ndcg_at_k(x, k))
/tmp/ipython-input-2408301555.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping col

Evaluando estrategia grupal: Least Misery...


/tmp/ipython-input-2408301555.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics = grouped_strategy.apply(lambda x: precision_recall_at_k(x, k))
/tmp/ipython-input-2408301555.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_scores = grouped_strategy.apply(lambda x: ndcg_at_k(x, k))
/tmp/ipython-input-2408301555.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping col

Evaluando estrategia grupal: Most Pleasure...


/tmp/ipython-input-2408301555.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics = grouped_strategy.apply(lambda x: precision_recall_at_k(x, k))
/tmp/ipython-input-2408301555.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_scores = grouped_strategy.apply(lambda x: ndcg_at_k(x, k))
/tmp/ipython-input-2408301555.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping col


--- Resultados de Evaluación (Estrategias Grupales) ---
        Strategy   K  Precision@K  Recall@K    nDCG@K  Novelty@K  Diversity@K
0        Average  10       0.8537  0.136536  0.945651  19.003976     0.769962
1   Least Misery  10       0.8551  0.136737  0.946098  19.007461     0.770752
2  Most Pleasure  10       0.8518  0.136273  0.943262  18.999800     0.776429

--- Comparativa Final: Individual vs. Grupal (nDCG@10) ---
DeepFM Individual: 0.8215
Average: 0.9457
Least Misery: 0.9461
Most Pleasure: 0.9433


/tmp/ipython-input-2408301555.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_diversity = grouped_strategy.apply(lambda x: diversity_at_k(x, k, df_mechanics)).mean()


Mostrar cómo sería las recomendaciones para cierto usuario o cierto grupo:

In [15]:
def recomendar_usuario(df_eval, user_id, k=10):
    """
    Muestra el Top-k de recomendaciones para un usuario dado,
    usando los scores de DeepFM.
    """
    user_scores = (
        df_eval[df_eval['userID'] == user_id]
        .sort_values('score', ascending=False)
        .head(k)
    )

    print(f"Recomendaciones para el usuario: {user_id}\n")
    return user_scores[['userID', 'itemID', 'score']]

# Ejemplo de uso:
# recomendar_usuario(df_eval, 12345, k=10)

def recomendar_grupo_average(df_eval, user_ids, k=10):
    """
    Hace recomendaciones grupales usando la estrategia de Average.
    user_ids: lista de ids de usuarios (por ejemplo, 4 usuarios).
    """
    df_grupo = df_eval[df_eval['userID'].isin(user_ids)]

    group_scores = (
        df_grupo.groupby('itemID')['score']
        .mean()
        .reset_index(name='score_grupo')
        .sort_values('score_grupo', ascending=False)
        .head(k)
    )

    print("Usuarios del grupo:", user_ids, "\n")
    print("Recomendaciones grupales (Average):\n")
    return group_scores[['itemID', 'score_grupo']]

# Ejemplo de uso:
# recomendar_grupo_average(df_eval, [u1, u2, u3, u4], k=10)


print("Caso para un usuario con ID: 235")
recomendar_usuario(df_eval, 235, k=10)
print("================================")

print("Caso para un grupo de usuarios con IDs: 235, 236, 237, 238")
recomendar_grupo_average(df_eval, [235, 236, 237, 238], k=10)
print("=================================")

Caso para un usuario con ID: 235
Recomendaciones para el usuario: 235

Caso para un grupo de usuarios con IDs: 235, 236, 237, 238
Usuarios del grupo: [235, 236, 237, 238] 

Recomendaciones grupales (Average):

